In [ ]:
from polygon import RESTClient
import os
import requests
import json
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta
from random import randrange
import time
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
class Polygon(object):
    def __init__(self,API_KEY):
        self.key = API_KEY
        self.client = RESTClient(self.key)
    
    def get_news(self,symbol):
        resp = self.client.reference_ticker_news(symbol)
        return resp
    
    def get_ticks(self,params):
        symbol,date,timestamp,timestampLimit,limit = params.values()
        tick_url = f'https://api.polygon.io/v2/ticks/stocks/trades/{symbol}/{date}?timestamp={timestamp}&timestampLimit={timestampLimit}&limit={limit}&apiKey={self.key}'
        print('tick_url',tick_url)
        r = requests.get(tick_url)
        ticks = r.json()
        return ticks
    
    def get_day_trades(self,symbol,start_date,unix_day_start,unix_day_end):
        results = []
#         elapsed_from_start = 0
        start_time = unix_day_start
        while True:
            print('elapsed_from_start',start_time - unix_day_start)
            params = {'symbol':symbol,'date':start_date,'timestamp':start_time,'timestampLimit':unix_day_end,'limit':50000}
            ticks = self.get_ticks(params)
            data = Polygon.store_data(ticks)
            if data.size > 0:
                results.append(data)
            if 'results' in ticks:
                if len(ticks['results']) < 50000:
                    break
                # Get new start
                temp = datetime.fromtimestamp(int(str(int(data[-1,0]))[:-9]))
                start_time = int(1e+9*(temp.timestamp()))
            else:
                break
        return results
        

    def get_range(self,symbol,start_date,end_date):
        """
        symbol : string
        start_date : string date
        end_date : string date
        """
        time_diff = datetime.strptime(end_date,'%Y-%m-%d') - datetime.strptime(start_date,'%Y-%m-%d')
        print(f'start date {start_date},end date {end_date},time span of {time_diff}')
        results = []
        i = 0
        while start_date != end_date:
            print('date',start_date)
#             start_time = datetime.strptime(start_date,'%Y-%m-%d') + timedelta(hours=9) + timedelta(minutes=30)
            temp_start = datetime.strptime(start_date,'%Y-%m-%d') + timedelta(hours=9) + timedelta(minutes=30)
            temp_end = datetime.strptime(start_date,'%Y-%m-%d') + timedelta(hours=16)
            unix_day_start = int(1e+9*(temp_start.timestamp()))
            unix_day_end = int(1e+9*(temp_end.timestamp()))

            day_trades = self.get_day_trades(symbol,start_date,unix_day_start,unix_day_end)
            results.append(day_trades)
            start_date = datetime.strptime(start_date,'%Y-%m-%d')
            start_date = str((start_date + timedelta(days=1)).date())
            
            if i > 100:
                print('loop break')
                break
            i += 1
        return results
    
    @staticmethod
    def store_data(ticks):
        if 'results' in ticks:
            print('num results',len(ticks['results']))
            data = np.empty((len(ticks['results']),3))
            for i,result in enumerate(ticks['results']):
                data[i,0] = result['t']
                data[i,1] = result['s']
                data[i,2] = result['p']
        else:
            data = np.array([])
        return data
    
    def get_symbol_data(self,symbol_list,start,end,num_samples):
        """
        symbol_list : list
        start : datetime object
        end : datetime object
        num_samples : int
        """
        stock_data = {}
        for symbol in symbol_list:
            symbol_results = []
            for _ in range(num_samples):
                random_start_date = random_date(start,end)
                start_date = datetime.strptime(str(random_start_date.date()),'%Y-%m-%d')
                
                string_start_date = str(start_date.date())
                string_end_date = str((start_date + timedelta(days=1)).date())
                
                results = self.get_range(symbol,string_start_date,string_end_date)
                symbol_results.append(results)
            stock_data[symbol] = symbol_results
        return stock_data
        
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)           
            

In [ ]:
# Insert API key here
API_KEY = ''

In [ ]:
interface = Polygon(API_KEY)

In [ ]:
# datetime(year, month, day, hour, minute, second, microsecond)
start = datetime(2017, 1, 1, 0, 0, 0, 0)
# datetime(year, month, day, hour, minute, second, microsecond)
end = datetime(2020, 1, 1, 0, 0, 0, 0)
start_date = '2018-01-01'
end_date = '2020-01-01'
# start = date(2018, 11, 28)
# end = date(2018, 11, 29)

# unix_start = start.timestamp()
# unix_end = int(1e+9*(end.timestamp())) 


In [ ]:
# test = str(datetime.fromtimestamp(int(str(int(unix_end))[:-9])).date)
# test
# print(datetime.strptime(start_date,'%Y-%m-%d'))
# day_end = datetime.strptime(start_date,'%Y-%m-%d') + timedelta(hours=9) + timedelta(minutes=30)
# print(day_end)

In [ ]:
symbol_list = ['TSLA','AAPL']
num_samples = 5

In [ ]:
results = interface.get_symbol_data(symbol_list,start,end,num_samples)

In [ ]:
print(results)

# Graph results

In [ ]:
def extract_x(data):
    X = np.array([])
    for sample in data:
        print(sample,len(sample[0]))
        if len(sample[0]) == 0:
            continue
        sample_X = sample[0][0][:,2]
        if X.size == 0:
            X = sample_X
        else:
            X = np.hstack((X,sample_X))
    return X

In [ ]:
tesla_results = results['TSLA']

In [ ]:
# print(tesla_results)

In [ ]:
X = extract_x(tesla_results)

In [ ]:
plt.plot(X)

In [ ]:
data = results[0][0]
print(datetime.fromtimestamp(int(str(int(data[0,0]))[:-9])))
print(datetime.fromtimestamp(int(str(int(data[-1,0]))[:-9])))

In [ ]:

string_date = datetime.strptime('2018-02-01','%Y-%m-%d')
print(string_date)
test_date = datetime.fromtimestamp(int(str(int(data[0,0]))[:-9]))
print(test_date)
test_date += timedelta(days=1)
print(test_date)

In [ ]:
resp = interface.get_news('tsla')

In [ ]:
resp.news

In [ ]:
content = []
sources = []
times = []
for i,new in enumerate(news):
#     print(dir(new))
#     print(new.title)
#     print(new.summary)
#     print(new.source)
#     print(new.keywords)
#     print(new.image)
#     print(new.timestamp)
    content.append(new.summary)
    sources.append(new.source)
    times.append(new.timestamp)

In [ ]:
np.unique(sources)

In [ ]:
print(content)

# Tick Data
 
- Timestamp -t
- size -s
- price -p

Sequentially get the tick data up to a certain time.

### Params
- start time
- end time

In [ ]:
start_time = date(2019, 4, 13)
end_time = date(2020,1,1)
# datetime(year, month, day, hour, minute, second, microsecond)
b = datetime(2017, 11, 28, 23, 55, 0, 0)

In [ ]:
data = np.empty((50000,3))
for i,result in enumerate(ticks['results']):
    data[i,0] = result['t']
    data[i,1] = result['s']
    data[i,2] = result['p']

In [ ]:
data[0,0] - data[1,0]

In [ ]:
type(int(data[0,0]))

In [ ]:
len(str(int(data[0,0])))

In [ ]:
test = str(int(data[0,0]))[:-9]

In [ ]:
print(datetime.fromtimestamp(int(str(int(data[0,0]))[:-9])))
print(datetime.fromtimestamp(int(str(int(data[-1,0]))[:-9])))

In [ ]:
datetime.fromtimestamp(int(str(int(data[0,0]))[:-9]))

In [ ]:
len(str(1577944970))

In [ ]:
datetime.fromtimestamp(1577944970)

In [ ]:
print(datetime.utcfromtimestamp(int(data[0,0])))

In [ ]:
dt_object = datetime.utcfromtimestamp(int(data[0,0]))

In [ ]:
data[0:10,:]

In [ ]:
data[-10:,:]

In [ ]:
keys = tickers.keys()

In [ ]:
ticker_names = tickers['tickers']

In [ ]:
names = [name['ticker'] for name in ticker_names]